In [1]:
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import text
import xlsxwriter
user = 'mirror_reader'
pwd = 'Xs5kgnl56ofc60t73K78Hvk9Y'
host = 'mirror-read-replica-rds-chemondis-bi.cxyit1wepgsg.eu-central-1.rds.amazonaws.com'
port = '10101'
db_name = 'mirror'



engine = create_engine(f'postgresql://{user}:{pwd}@{host}:{port}/{db_name}')
connection = engine.connect()

In [2]:

query = '''SELECT
    company_mart."name"  AS "company_name",
    company_mart."company_id"  AS "company_id",
    business_owner_mart."name"  AS "Account Owner",
    product_substance_mart."cas_number"  AS "Substance cas_number",
    product_substance_mart."name"  AS "substance_name"
FROM vault.product_mart  AS product_mart
INNER JOIN vault.company_mart  AS company_mart ON (product_mart."company_id_hash_key") = (company_mart."company_id_hash_key")
LEFT JOIN vault.business_owner_mart  AS business_owner_mart ON (company_mart."owner_id_hash_key") = (case when not (business_owner_mart."is_bd_active") then null else business_owner_mart."owner_id_hash_key" end)
LEFT JOIN vault.product_substance_mart  AS product_substance_mart ON (product_substance_mart."product_id_hash_key") = (product_mart."product_id_hash_key")
WHERE (NOT (product_mart."is_supplier_lanxess") OR (product_mart."is_supplier_lanxess") IS NULL) AND (NOT (((company_mart."deleted") is not null) or (((company_mart."name") ~* 'delete')) or ((length((company_mart."name"))) <= 2)) OR (((company_mart."deleted") is not null) or (((company_mart."name") ~* 'delete')) or ((length((company_mart."name"))) <= 2)) IS NULL) AND (NOT (company_mart."is_lanxess") OR (company_mart."is_lanxess") IS NULL) AND (NOT ((DATE(product_substance_mart."deleted" )) is not null ) OR ((DATE(product_substance_mart."deleted" )) is not null ) IS NULL)
GROUP BY
    1,
    2,
    3,
    4,
    5
ORDER BY
    1'''
company_substances = pd.read_sql(text(query), connection)
company_substances

In [3]:
query2 = '''select z."company_mart.name"                                                 as company,
       z."company_mart.has_active_trial_supplier_membership"                 as in_trial,
       z."company_mart.has_signed_supplier_membership_contract"              as has_signed_up,
       sum(coalesce(z."count_of_product_name", 0))                           as company_top_30_product_count,                             -- number of products that this Supplier has
       sum(coalesce(z."product_mart.total_number_of_requests", 0))           as company_top_30_request_count,                             -- number of requests that this Supplier received
       sum(coalesce(z."negotiation_mart.number_of_requests", 0))             as total_substance_request_count_based_on_company_substances, -- the market addressed by the respective company
       sum(coalesce(z."negotiation_history_mart.total_number_of_offers", 0)) as total_substance_offers_count_based_on_company_substances,  -- total number of offers for this Substance on the marketplace

       round((sum(coalesce(z."product_mart.total_number_of_requests", 0)) /
              sum(coalesce(z."count_of_product_name", 0)))::numeric)         as avg_requests_per_product,                                  -- for the particular Supplier
       case
           when sum(coalesce(z."negotiation_mart.number_of_requests", 0)) = 0 then -1
           else
               round((sum(coalesce(z."product_mart.total_number_of_requests", 0)) /
                      sum(coalesce(z."negotiation_mart.number_of_requests", 0)))::numeric,
                     2) end                                                  as market_share_requests,                                     -- (-1) means that this substance has never been requested on the marketplace

       round((sum(coalesce(z."negotiation_history_mart.total_number_of_offers", 0)) /
              sum(coalesce(z."negotiation_mart.number_of_requests", 1)))::numeric,
             2)                                                              as market_request_to_offer_cr  ,
       -- based on the substances the company lists, what is the conversion rate to offers

       array_agg(z."product_substance_mart.cas_number")                      as cas_list                                                   -- cas numbers of the products that this Supplier has

       -- market share percentag
       -- offer conversion rate for the company in their specific markeet
       --
       -- eventually I''d like to have a blunt percentage which indicates the "probability of success" on our marketplace based on the actual product portfolio
       -- tech wise i imagine this to become just a database view, so we can quickly starting to work with this in looker. No need for a deeper integration into the vault in the first step

from (
         (SELECT company_mart."name"                   AS "company_mart.name",                    -- company name
                 business_owner_mart."name"            AS "business_owner_mart.name",
                 product_substance_mart."cas_number"   AS "product_substance_mart.cas_number",    -- cas number
                 (CASE
                      WHEN company_mart."has_active_supplier_membership" THEN 'Yes'
                      ELSE 'No' END)                   AS "company_mart.has_active_supplier_membership",
                 (CASE
                      WHEN company_mart."has_active_trial_supplier_membership" THEN 'Yes'
                      ELSE 'No' END)                   AS "company_mart.has_active_trial_supplier_membership",
                 (CASE
                      WHEN company_mart."has_signed_supplier_membership_contract" THEN 'Yes'
                      ELSE 'No' END)                   AS "company_mart.has_signed_supplier_membership_contract",
                 COUNT(DISTINCT (product_mart."name")) AS "count_of_product_name",
                 COALESCE(CAST((SUM(DISTINCT
                                    (CAST(FLOOR(COALESCE((product_mart."number_of_requests"), 0) * (1000000 * 1.0)) AS DECIMAL(65, 0))) +
                                    ('x' || MD5(product_mart."product_id_hash_key" ::varchar))::bit(64)::bigint::DECIMAL(65, 0) *
                                    18446744073709551616 +
                                    ('x' || SUBSTR(MD5(product_mart."product_id_hash_key" ::varchar), 17))::bit(64)::bigint::DECIMAL(65, 0)) -
                                SUM(DISTINCT
                                    ('x' || MD5(product_mart."product_id_hash_key" ::varchar))::bit(64)::bigint::DECIMAL(65, 0) *
                                    18446744073709551616 +
                                    ('x' || SUBSTR(MD5(product_mart."product_id_hash_key" ::varchar), 17))::bit(64)::bigint::DECIMAL(65, 0))) AS DOUBLE PRECISION) /
                          CAST((1000000 * 1.0) AS DOUBLE PRECISION),
                          0)                           AS "product_mart.total_number_of_requests" -- number of requests of this product from this particular Supplier
          FROM vault.product_mart AS product_mart
                   INNER JOIN vault.company_mart AS company_mart
                              ON (product_mart."company_id_hash_key") = (company_mart."company_id_hash_key")
                   LEFT JOIN vault.business_owner_mart AS business_owner_mart ON (company_mart."owner_id_hash_key") =
                                                                                 (case
                                                                                      when not (business_owner_mart."is_bd_active")
                                                                                          then null
                                                                                      else business_owner_mart."owner_id_hash_key" end)
                   LEFT JOIN vault.product_substance_mart AS product_substance_mart
                             ON (product_substance_mart."product_id_hash_key") = (product_mart."product_id_hash_key")
          WHERE (NOT (case when (product_mart."deleted") is null then false else true end) OR
                 (case when (product_mart."deleted") is null then false else true end) IS NULL)
            AND (NOT (product_mart."is_lanxess") OR (product_mart."is_lanxess") IS NULL)
            AND (company_mart."number_of_products") >= 1
            AND (product_substance_mart."cas_number") IS NOT NULL
          GROUP BY 1, 2, 3, 4, 5, 6
          ORDER BY 7 DESC) x
             left join
             (SELECT product_substance_mart."cas_number"                          AS "product_substance_mart.cas_number_2",   -- product cas number
                     product_substance_mart."name"                                AS "product_substance_mart.substance_name", -- product substance name
                     COUNT(DISTINCT (negotiation_mart."negotiation_id_hash_key")) AS "negotiation_mart.number_of_requests",   -- how many times this cas_number was requested from all suppliers?
                     COALESCE(CAST((SUM(DISTINCT
                                        (CAST(FLOOR(COALESCE((negotiation_history_mart."number_of_offers"), 0) *
                                                    (1000000 * 1.0)) AS DECIMAL(65, 0))) + ('x' ||
                                                                                            MD5(negotiation_history_mart."negotiation_id_hash_key" ::varchar))::bit(64)::bigint::DECIMAL(65, 0) *
                                                                                           18446744073709551616 +
                                        ('x' ||
                                         SUBSTR(MD5(negotiation_history_mart."negotiation_id_hash_key" ::varchar),
                                                17))::bit(64)::bigint::DECIMAL(65, 0)) - SUM(DISTINCT ('x' ||
                                                                                                       MD5(negotiation_history_mart."negotiation_id_hash_key" ::varchar))::bit(64)::bigint::DECIMAL(65, 0) *
                                                                                                      18446744073709551616 +
                                                                                                      ('x' || SUBSTR(
                                                                                                              MD5(negotiation_history_mart."negotiation_id_hash_key" ::varchar),
                                                                                                              17))::bit(64)::bigint::DECIMAL(65, 0))) AS DOUBLE PRECISION) /
                              CAST((1000000 * 1.0) AS DOUBLE PRECISION),
                              0)                                                  AS "negotiation_history_mart.total_number_of_offers"
              FROM vault.negotiation_mart AS negotiation_mart
                       LEFT JOIN vault.company_mart AS company_mart_supplier
                                 ON (company_mart_supplier."company_id_hash_key") =
                                    (negotiation_mart."seller_id_hash_key")
                       LEFT JOIN vault.product_mart AS product_mart
                                 ON (product_mart."product_id_hash_key") = (negotiation_mart."product_id_hash_key")
                       LEFT JOIN vault.product_substance_mart AS product_substance_mart
                                 ON (product_mart."product_id_hash_key") =
                                    (product_substance_mart."product_id_hash_key")
                       LEFT JOIN vault.negotiation_history_mart AS negotiation_history_mart
                                 ON (negotiation_mart."negotiation_id_hash_key") =
                                    (negotiation_history_mart."negotiation_id_hash_key")
              WHERE (NOT (((company_mart_supplier."deleted") is not null) or
                          (((company_mart_supplier."name") ~* 'delete')) or
                          ((length((company_mart_supplier."name"))) <= 2)) OR
                     (((company_mart_supplier."deleted") is not null) or
                      (((company_mart_supplier."name") ~* 'delete')) or
                      ((length((company_mart_supplier."name"))) <= 2)) IS NULL)
                AND (company_mart_supplier."verified")
                AND (NOT (company_mart_supplier."is_lanxess") OR (company_mart_supplier."is_lanxess") IS NULL)
                AND (product_substance_mart."cas_number") IS NOT NULL
              GROUP BY 1, 2
              HAVING COUNT(DISTINCT (negotiation_mart."negotiation_id_hash_key")) > 0
              ORDER BY 3 DESC
              LIMIT 30) y
         on x."product_substance_mart.cas_number" = y."product_substance_mart.cas_number_2"
         ) z
group by company, in_trial, has_signed_up
order by "total_substance_request_count_based_on_company_substances" desc'''

In [4]:
scores = pd.read_sql(text(query2), connection)
scores

,company,in_trial,has_signed_up,company_top_30_product_count,company_top_30_request_count,total_substance_request_count_based_on_company_substances,total_substance_offers_count_based_on_company_substances,avg_requests_per_product,market_share_requests,market_request_to_offer_cr,cas_list
0,Hanke + Seidel GmbH & Co. KG,No,No,399.0,326.0,30504.0,8286.0,1.0,0.01,0.27,"[7664-93-9, 10043-52-4, 64365-11-3, 10192-30-0..."
1,AG Chemi Group,No,No,2424.0,457.0,29865.0,8207.0,0.0,0.02,0.27,"[11118-57-3, 11103-86-9, 11094-60-3, 110-91-8,..."
2,Mikro Teknik Kimyevi Maddeler Lab. Malz. Cihaz...,No,No,174.0,430.0,29650.0,7948.0,2.0,0.01,0.27,"[7664-39-3, 7664-93-9, 7681-38-1, 7681-52-9, 7..."
3,neoFroxx GmbH,No,Yes,1357.0,387.0,29169.0,7985.0,0.0,0.01,0.27,"[7775-14-6, 606-68-8, 609-99-4, 6100-05-6, 100..."
4,Molekula GmbH,No,Yes,4238.0,2013.0,29129.0,8046.0,0.0,0.07,0.24,"[27164-46-1, 860-22-0, 86-00-0, 859-18-7, 85-8..."
...,...,...,...,...,...,...,...,...,...,...,...
1514,Productos Albi,No,No,6.0,9.0,0.0,0.0,2.0,-1.00,0.00,"[10043-01-3, 10043-67-1, 7784-25-0]"
1515,Pro-Eko Srl,No,No,7.0,23.0,0.0,0.0,3.0,-1.00,0.00,"[112-00-5, 17372-87-1, 25655-41-8, 517-28-2, 5..."
1516,PROLIFE BIO-CHEMICAL INDUSTRIES,No,No,43.0,12.0,0.0,0.0,0.0,-1.00,0.00,"[89-62-3, 506-87-6, 5307-03-9, 502453-61-4, 58..."
1517,PROREUSE SRLS,No,No,13.0,97.0,0.0,0.0,7.0,-1.00,0.00,"[8006-40-4, 92045-76-6, 63231-60-7, 64742-42-3..."


In [5]:
query3 = '''SELECT
    product_substance_mart."name"  AS "substance_name",
    product_substance_mart."cas_number"  AS "Substance cas_number",
    COALESCE(CAST( ( SUM(DISTINCT (CAST(FLOOR(COALESCE( ( product_mart."number_of_requests"  )  ,0)*(1000000*1.0)) AS DECIMAL(65,0))) + ('x' || MD5( product_mart."product_id_hash_key"  ::varchar))::bit(64)::bigint::DECIMAL(65,0)  *18446744073709551616 + ('x' || SUBSTR(MD5( product_mart."product_id_hash_key"  ::varchar),17))::bit(64)::bigint::DECIMAL(65,0) ) - SUM(DISTINCT ('x' || MD5( product_mart."product_id_hash_key"  ::varchar))::bit(64)::bigint::DECIMAL(65,0)  *18446744073709551616 + ('x' || SUBSTR(MD5( product_mart."product_id_hash_key"  ::varchar),17))::bit(64)::bigint::DECIMAL(65,0)) )  AS DOUBLE PRECISION) / CAST((1000000*1.0) AS DOUBLE PRECISION), 0) AS "total_number_of_requests"
FROM vault.product_mart  AS product_mart
INNER JOIN vault.company_mart  AS company_mart ON (product_mart."company_id_hash_key") = (company_mart."company_id_hash_key")
LEFT JOIN vault.product_substance_mart  AS product_substance_mart ON (product_substance_mart."product_id_hash_key") = (product_mart."product_id_hash_key")
WHERE (NOT (case when (product_mart."deleted") is null then false else true end) OR (case when (product_mart."deleted") is null then false else true end) IS NULL) AND ((NOT (product_mart."is_lanxess" ) OR (product_mart."is_lanxess" ) IS NULL) AND (NOT (product_mart."is_supplier_lanxess") OR (product_mart."is_supplier_lanxess") IS NULL)) AND ((product_mart."is_active") AND ((NOT (((company_mart."deleted") is not null) or (((company_mart."name") ~* 'delete')) or ((length((company_mart."name"))) <= 2)) OR (((company_mart."deleted") is not null) or (((company_mart."name") ~* 'delete')) or ((length((company_mart."name"))) <= 2)) IS NULL) AND (product_substance_mart."cas_number" ) IS NOT NULL))
GROUP BY
    1,
    2
ORDER BY
    3 DESC
FETCH NEXT 30 ROWS ONLY'''

In [6]:
top30_data = pd.read_sql(text(query3), connection)
top30_data

,substance_name,Substance cas_number,total_number_of_requests
0,Urea,57-13-6,939.0
1,Isopropanol,67-63-0,790.0
2,Sodium Hydroxide,1310-73-2,741.0
3,Phosphoric Acid,7664-38-2,598.0
4,Propylene Glycol,57-55-6,584.0
5,Glycerol,56-81-5,557.0
6,Acetone,67-64-1,534.0
7,Ethanol,64-17-5,498.0
8,Hydrogen Peroxide,7722-84-1,433.0
9,Citric Acid Monohydrate,5949-29-1,418.0


In [7]:
top30 = top30_data['substance_name']
top30 = list(top30)
top30

['Urea',
 'Isopropanol',
 'Sodium Hydroxide',
 'Phosphoric Acid',
 'Propylene Glycol',
 'Glycerol',
 'Acetone',
 'Ethanol',
 'Hydrogen Peroxide',
 'Citric Acid Monohydrate',
 'Methylene Chloride',
 'Citric Acid',
 'Titanium Dioxide',
 'Ethyl Acetate',
 'Hydrogen Chloride',
 'Ethane-1,2-Diol',
 '2-Butoxyethanol',
 'N-Butyl Acetate',
 'Potassium Hydroxide',
 'Acetic Acid',
 'Xylene',
 'Sodium Bicarbonate',
 'Sodium Carbonate',
 'Methyl Ethyl Ketone',
 '2-(2-Butoxyethoxy)Ethanol',
 'Ethanolamine',
 'Methanol',
 'Polyethylene Glycol (Peg)',
 'Xanthan Gum',
 'Sulfuric Acid']

In [8]:
top30_substances_requests = top30_data[['substance_name','total_number_of_requests']]
top30_substances_requests

,substance_name,total_number_of_requests
0,Urea,939.0
1,Isopropanol,790.0
2,Sodium Hydroxide,741.0
3,Phosphoric Acid,598.0
4,Propylene Glycol,584.0
5,Glycerol,557.0
6,Acetone,534.0
7,Ethanol,498.0
8,Hydrogen Peroxide,433.0
9,Citric Acid Monohydrate,418.0


In [9]:
company_substances['flag'] = company_substances['substance_name'].isin(top30).astype(int)
company_substances

,company_name,company_id,Account Owner,Substance cas_number,substance_name,flag
0,20 Microns Limited,3896,None,12001-26-2,Mica,0
1,20 Microns Limited,3896,None,1309-37-1,Iron (Iii) Oxide,0
2,20 Microns Limited,3896,None,1332-58-7,Kaolin,0
3,20 Microns Limited,3896,None,13462-86-7,Barite (Ba(SO4)),0
4,20 Microns Limited,3896,None,14807-96-6,Magnesium Silicate Monohydrate (Talc),0
...,...,...,...,...,...,...
99485,Хим Органик,39346,None,7647-14-5,Sodium Chloride,0
99486,Химпищеаромат (Khimpishchearomat LLC),9519,Oksana Senchyshyn,5329-14-6,Sulphamidic Acid,0
99487,Химпищеаромат (Khimpishchearomat LLC),9519,Oksana Senchyshyn,64-17-5,Ethanol,1
99488,Химпищеаромат (Khimpishchearomat LLC),9519,Oksana Senchyshyn,69-72-7,Salicylic Acid,0


In [10]:
df = pd.merge(company_substances, top30_substances_requests, on='substance_name',how="left")


In [11]:
df['total_number_of_requests'] = df['total_number_of_requests'].fillna(0)


In [12]:
account_owners = '''SELECT
    company_mart."name"  AS "company_name",
    company_mart."company_id"  AS "company_id",
    business_owner_mart."name"  AS "account_owner"
FROM vault.product_mart  AS product_mart
INNER JOIN vault.company_mart  AS company_mart ON (product_mart."company_id_hash_key") = (company_mart."company_id_hash_key")
LEFT JOIN vault.business_owner_mart  AS business_owner_mart ON (company_mart."owner_id_hash_key") = (case when not (business_owner_mart."is_bd_active") then null else business_owner_mart."owner_id_hash_key" end)
GROUP BY
    1,
    2,
    3
ORDER BY
    1'''

account_owners = pd.read_sql(text(account_owners), connection)
account_owners

,company_name,company_id,account_owner
0,20 Microns Limited,3896,None
1,23street I solutions,61618,None
2,2D KIMYA SANAYI,14123,Tunahan Güzel
3,2M-SOLUTION GmbH,60458,Florian Pagels
4,3E AKANTOS TRATOS,27904,None
...,...,...,...
2253,ZS Latvia bee,12317,None
2254,"ZÜLFİKARLAR HOLDİNG, AKÇA KİMYA",22588,Tunahan Güzel
2255,ип Шумеков,62829,None
2256,Хим Органик,39346,None


In [13]:
df_flag = df[['company_id',"company_name",'flag']]
df_reqeust_score = df[['company_id',"company_name",'total_number_of_requests']]

In [14]:
match_count = df_flag.groupby('company_id',as_index=False)['flag'].sum()
match_count

,company_id,flag
0,1001,11
1,10016,0
2,10088,7
3,101,1
4,10106,0
...,...,...
2008,9806,0
2009,984,1
2010,9891,0
2011,991,1


In [50]:
request_score = df_reqeust_score.pivot_table(columns=['company_id','company_name'], values='total_number_of_requests', aggfunc='sum').sum().reset_index()
request_score_density = df_reqeust_score.pivot_table(columns=['company_id','company_name'], values='total_number_of_requests', aggfunc='mean').mean().reset_index()
product_density = df_flag.pivot_table(columns=['company_id','company_name'], values='flag', aggfunc='mean').mean().reset_index()


In [53]:
product_density = product_density.rename(columns={0: 'Product Density Score'})
request_score = request_score.rename(columns={0: 'Requests Score'})
request_score_density =request_score_density.rename(columns={0: 'Requests Density Score'})


In [55]:
profile1 = product_density.merge(request_score,on=['company_id','company_name'])


,company_id,company_name,Product Density Score,Requests Score
0,1001,Balthemis,0.687500,5623.0
1,10016,Polymer Corp LLC,0.000000,0.0
2,10088,MRV Mineral- und Rohstoffvertrieb GmbH,0.636364,3191.0
3,101,Harold Scholz & Co. GmbH,0.066667,376.0
4,10106,Samyang Biophamraceuticals Corp.,0.000000,0.0
...,...,...,...,...
2008,9806,SAFIC-ALCAN RUS,0.000000,0.0
2009,984,Bozzetto Group,0.111111,790.0
2010,9891,Cosmetica Bolognese,0.000000,0.0
2011,991,Gama (York) Ltd,0.500000,790.0


In [56]:
profile = profile1.merge(request_score_density,on=['company_id','company_name'])
profile=profile.rename(columns={"company_name": 'company'})


,company_id,company,Product Density Score,Requests Score,Requests Density Score
0,1001,Balthemis,0.687500,5623.0,351.437500
1,10016,Polymer Corp LLC,0.000000,0.0,0.000000
2,10088,MRV Mineral- und Rohstoffvertrieb GmbH,0.636364,3191.0,290.090909
3,101,Harold Scholz & Co. GmbH,0.066667,376.0,25.066667
4,10106,Samyang Biophamraceuticals Corp.,0.000000,0.0,0.000000
...,...,...,...,...,...
2008,9806,SAFIC-ALCAN RUS,0.000000,0.0,0.000000
2009,984,Bozzetto Group,0.111111,790.0,87.777778
2010,9891,Cosmetica Bolognese,0.000000,0.0,0.000000
2011,991,Gama (York) Ltd,0.500000,790.0,395.000000


In [57]:
profile["Count of Substances in Top30"] = match_count.flag


,company_id,company,Product Density Score,Requests Score,Requests Density Score,Count of Substances in Top30
0,1001,Balthemis,0.687500,5623.0,351.437500,11
1,10016,Polymer Corp LLC,0.000000,0.0,0.000000,0
2,10088,MRV Mineral- und Rohstoffvertrieb GmbH,0.636364,3191.0,290.090909,7
3,101,Harold Scholz & Co. GmbH,0.066667,376.0,25.066667,1
4,10106,Samyang Biophamraceuticals Corp.,0.000000,0.0,0.000000,0
...,...,...,...,...,...,...
2008,9806,SAFIC-ALCAN RUS,0.000000,0.0,0.000000,0
2009,984,Bozzetto Group,0.111111,790.0,87.777778,1
2010,9891,Cosmetica Bolognese,0.000000,0.0,0.000000,0
2011,991,Gama (York) Ltd,0.500000,790.0,395.000000,1


In [58]:
merged_scores = scores.merge(profile, on='company', how='left')


,company,in_trial,has_signed_up,company_top_30_product_count,company_top_30_request_count,total_substance_request_count_based_on_company_substances,total_substance_offers_count_based_on_company_substances,avg_requests_per_product,market_share_requests,market_request_to_offer_cr,cas_list,company_id,Product Density Score,Requests Score,Requests Density Score,Count of Substances in Top30
0,Hanke + Seidel GmbH & Co. KG,No,No,399.0,326.0,30504.0,8286.0,1.0,0.01,0.27,"[7664-93-9, 10043-52-4, 64365-11-3, 10192-30-0...",19556,0.068966,12059.0,27.721839,30.0
1,AG Chemi Group,No,No,2424.0,457.0,29865.0,8207.0,0.0,0.02,0.27,"[11118-57-3, 11103-86-9, 11094-60-3, 110-91-8,...",51976,0.034307,11202.0,14.233799,27.0
2,Mikro Teknik Kimyevi Maddeler Lab. Malz. Cihaz...,No,No,174.0,430.0,29650.0,7948.0,2.0,0.01,0.27,"[7664-39-3, 7664-93-9, 7681-38-1, 7681-52-9, 7...",19413,0.189189,11452.0,77.378378,28.0
3,neoFroxx GmbH,No,Yes,1357.0,387.0,29169.0,7985.0,0.0,0.01,0.27,"[7775-14-6, 606-68-8, 609-99-4, 6100-05-6, 100...",589,0.040000,11052.0,17.003077,26.0
4,Molekula GmbH,No,Yes,4238.0,2013.0,29129.0,8046.0,0.0,0.07,0.24,"[27164-46-1, 860-22-0, 86-00-0, 859-18-7, 85-8...",4794,0.006924,11523.0,2.849407,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,Productos Albi,No,No,6.0,9.0,0.0,0.0,2.0,-1.00,0.00,"[10043-01-3, 10043-67-1, 7784-25-0]",24051,0.000000,0.0,0.000000,0.0
1520,Pro-Eko Srl,No,No,7.0,23.0,0.0,0.0,3.0,-1.00,0.00,"[112-00-5, 17372-87-1, 25655-41-8, 517-28-2, 5...",2185,0.000000,0.0,0.000000,0.0
1521,PROLIFE BIO-CHEMICAL INDUSTRIES,No,No,43.0,12.0,0.0,0.0,0.0,-1.00,0.00,"[89-62-3, 506-87-6, 5307-03-9, 502453-61-4, 58...",6366,0.000000,0.0,0.000000,0.0
1522,PROREUSE SRLS,No,No,13.0,97.0,0.0,0.0,7.0,-1.00,0.00,"[8006-40-4, 92045-76-6, 63231-60-7, 64742-42-3...",11160,0.000000,0.0,0.000000,0.0


In [59]:
merged = merged_scores.merge(account_owners, on='company_id', how='left')



,company,in_trial,has_signed_up,company_top_30_product_count,company_top_30_request_count,total_substance_request_count_based_on_company_substances,total_substance_offers_count_based_on_company_substances,avg_requests_per_product,market_share_requests,market_request_to_offer_cr,cas_list,company_id,Product Density Score,Requests Score,Requests Density Score,Count of Substances in Top30,company_name,account_owner
0,Hanke + Seidel GmbH & Co. KG,No,No,399.0,326.0,30504.0,8286.0,1.0,0.01,0.27,"[7664-93-9, 10043-52-4, 64365-11-3, 10192-30-0...",19556,0.068966,12059.0,27.721839,30.0,Hanke + Seidel GmbH & Co. KG,Julian Panchyrz
1,AG Chemi Group,No,No,2424.0,457.0,29865.0,8207.0,0.0,0.02,0.27,"[11118-57-3, 11103-86-9, 11094-60-3, 110-91-8,...",51976,0.034307,11202.0,14.233799,27.0,AG Chemi Group,Oksana Senchyshyn
2,Mikro Teknik Kimyevi Maddeler Lab. Malz. Cihaz...,No,No,174.0,430.0,29650.0,7948.0,2.0,0.01,0.27,"[7664-39-3, 7664-93-9, 7681-38-1, 7681-52-9, 7...",19413,0.189189,11452.0,77.378378,28.0,Mikro Teknik Kimyevi Maddeler Lab. Malz. Cihaz...,Tunahan Güzel
3,neoFroxx GmbH,No,Yes,1357.0,387.0,29169.0,7985.0,0.0,0.01,0.27,"[7775-14-6, 606-68-8, 609-99-4, 6100-05-6, 100...",589,0.040000,11052.0,17.003077,26.0,neoFroxx GmbH,Daniel Wigan
4,Molekula GmbH,No,Yes,4238.0,2013.0,29129.0,8046.0,0.0,0.07,0.24,"[27164-46-1, 860-22-0, 86-00-0, 859-18-7, 85-8...",4794,0.006924,11523.0,2.849407,28.0,Molekula GmbH,Florian Pagels
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,Productos Albi,No,No,6.0,9.0,0.0,0.0,2.0,-1.00,0.00,"[10043-01-3, 10043-67-1, 7784-25-0]",24051,0.000000,0.0,0.000000,0.0,Productos Albi,Carlos Nucete
1520,Pro-Eko Srl,No,No,7.0,23.0,0.0,0.0,3.0,-1.00,0.00,"[112-00-5, 17372-87-1, 25655-41-8, 517-28-2, 5...",2185,0.000000,0.0,0.000000,0.0,Pro-Eko Srl,Rovena Bakaj
1521,PROLIFE BIO-CHEMICAL INDUSTRIES,No,No,43.0,12.0,0.0,0.0,0.0,-1.00,0.00,"[89-62-3, 506-87-6, 5307-03-9, 502453-61-4, 58...",6366,0.000000,0.0,0.000000,0.0,PROLIFE BIO-CHEMICAL INDUSTRIES,Oksana Senchyshyn
1522,PROREUSE SRLS,No,No,13.0,97.0,0.0,0.0,7.0,-1.00,0.00,"[8006-40-4, 92045-76-6, 63231-60-7, 64742-42-3...",11160,0.000000,0.0,0.000000,0.0,PROREUSE SRLS,Rovena Bakaj


In [63]:
df2 = merged[['company_id','company', 'in_trial', 'has_signed_up', 'account_owner',
       'avg_requests_per_product', 'market_share_requests',
       'market_request_to_offer_cr','Requests Score',
       'Product Density Score', 'Requests Density Score',
       'Count of Substances in Top30']]

In [64]:
# Create an ExcelWriter object and specify the filename
writer = pd.ExcelWriter('Potential Members.xlsx', engine='xlsxwriter')

# Write each dataframe to a separate sheet in the file
df2.to_excel(writer, sheet_name='Scores')
merged.to_excel(writer, sheet_name='All Scores')

# Save the file
writer.save()

In [30]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
